In [1]:
cd tutorial

C:\Users\LENOVO\Practical_Natural_Language_Processing\tutorial


In [2]:
import scrapy

class BookSpider(scrapy.Spider):
    name = "books"

    start_urls = ["http://books.toscrape.com/"]

In [3]:
import scrapy

class BookSpider(scrapy.Spider):
    name = "books"

    start_urls = ["http://books.toscrape.com/"]
    def parse(self, response):
        print(response.url)

In [6]:
!pip install IPython

In [10]:
response= "file:///C:/Users/LENOVO/AppData/Local/Temp/tmpvptt9lp5.html"

In [19]:
class BookSpider(scrapy.Spider):
    name = "books"

    start_urls = ["http://books.toscrape.com/"]
    response= "file:///C:/Users/LENOVO/AppData/Local/Temp/tmpvptt9lp5.html"
    def parse(self, response):
        num_cats_to_parse = 5
        cat_names = response.xpath("//div[@class='side_categories']/ul/li/ul/li/a/text()").getall()
        cat_urls = response.xpath("//div[@class='side_categories']/ul/li/ul/li/a/@href").getall()
        for _, name, url in zip(range(num_cats_to_parse), cat_names, cat_urls):
            name = name.strip()
            url = response.urljoin(url)
            yield scrapy.Request(url,
                                 callback=self.parse_category,
                                 cb_kwargs=dict(cat_name=name))
    def parse_category(self, response, name):
        book_urls = response.xpath("//article[@class='product_pod']/h3/a/@href").getall()

        for book_url in book_urls:
            book_url = response.urljoin(book_url)
            yield scrapy.Request(book_url, callback=self.parse_book,
                                 cb_kwargs=dict(cat_name=name))

        next_button = response.css(".next a")
        if next_button:
            next_url = next_button.attrib["href"]
            next_url = response.urljoin(next_url)
            yield scrapy.Request(next_url,
                                 callback=self.parse_category,
                                 cb_kwargs=dict(cat_name=name))
    def parse_book(self,response,name):
        title_names=response.xpath("//instock[@class='title']/ul/li/ul/li/a/text()").getall()
        title_urls=response.xpath("//instock[@class='title']/ul/li/ul/li/a/@href").getall()
        price_names=response.xpath("//instock[@class='price']/ul/li/ul/li/a/text()").getall()
        price_urls=response.xpath("//instock[@class='price']/ul/li/ul/li/a/@href").getall()
        instock_names=response.xpath("//instock[@class='stock_information']/ul/li/ul/li/a/text()").getall()
        instock_urls=response.xpath("//instock[@class='stock_information']/ul/li/ul/li/a/@href").getall()
        star_names=response.xpath("//instock[contains(@class, 'star_rating')]/h2/a/text()").getall()
        star_urls=response.xpath("//instock[contains(@class, 'star_rating')]/h2/a/@href").getall()
        for  t_name, t_url,p_name,p_url,i_name,i_url,s_name,s_url in zip(title_names, title_urls, price_names,price_urls,instock_names,instock_urls,star_names,star_urls):
            t_url = response.urljoin(t_url)
            p_url = response.urljoin(p_url)
            i_name = i_name.strip()
            i_url = response.urljoin(i_url)
            s_url = response.urljoin(s_url)
            yield scrapy.Request([t_url,p_url,i_url,s_url],
                                 cb_kwargs=dict(cat_name=name))

        next_button = response.css(".next a")
        if next_button:
            next_url = next_button.attrib["href"]
            next_url = response.urljoin(next_url)
            yield scrapy.Request(next_url,
                                 callback=self.parse_book,
                                 cb_kwargs=dict(cat_name=name))

In [20]:
c=BookSpider()
c.parse

<bound method BookSpider.parse of <BookSpider 'books' at 0x1b8493140a0>>

In [21]:
c.parse_category

<bound method BookSpider.parse_category of <BookSpider 'books' at 0x1b8493140a0>>

In [22]:
c.parse_book

<bound method BookSpider.parse_book of <BookSpider 'books' at 0x1b8493140a0>>

In [24]:
import csv

class BookCsvPipeline():
    def open_spider(self, spider):
        self.file = open("output.csv", "at")
        fieldnames = ["title",
                      "price",
                      "stock",
                      "rating",
                      "category"]
        self.writer = csv.DictWriter(self.file, fieldnames=fieldnames)
        if self.file.tell() == 0:
            self.writer.writeheader()

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        self.writer.writerow(item)
        return item

In [25]:
d=BookCsvPipeline()

In [26]:
d.open_spider(c)

In [27]:
d.close_spider(c)

In [28]:
d.process_item(c.parse_book,c)

AttributeError: 'function' object has no attribute 'keys'